#### Query the articles

In [1]:
import logging
import numpy as np
import ollama
import time
import json
import chromadb
from typing import List, Dict, Optional, Tuple

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuration
db_path = './chroma_db'  # Directory for Chroma persistent storage
collection_name = 'articles'
EMBEDDING_DIM = 768  # Dimension of nomic-embed-text embeddings

# In-memory data stores
articles_memory = []  # Stores all article data
embeddings_memory = None  # Stores all embeddings as numpy array

# Conversation history
conversation_history = []
MAX_HISTORY_LENGTH = 5  # Maximum number of recent exchanges to keep

def load_articles_to_memory():
    """Load all articles and embeddings into memory from Chroma at startup"""
    global articles_memory, embeddings_memory
    
    try:
        # Initialize Chroma client
        client = chromadb.PersistentClient(path=db_path)
        collection = client.get_collection(collection_name)
        
        # Load all articles
        results = collection.get(include=['embeddings', 'metadatas', 'documents'])
        
        # Prepare data structures
        articles_memory = []
        embeddings_list = []
        
        for idx, (doc_id, embedding, metadata, document) in enumerate(
            zip(results['ids'], results['embeddings'], results['metadatas'], results['documents'])
        ):
            article = {
                'id': int(metadata.get('id', idx + 1)),
                'title': metadata.get('title', 'Unknown'),
                'url': metadata.get('url', 'Unknown'),
                'full_text': document,
                'publish_date': metadata.get('publish_date', 'Unknown'),
                'keyword': metadata.get('keyword', 'Unknown'),
                'author': metadata.get('author', 'Unknown'),
                'article_keywords': json.loads(metadata.get('article_keywords', '[]'))
            }
            articles_memory.append(article)
            embeddings_list.append(np.array(embedding, dtype=np.float32))
        
        embeddings_memory = np.array(embeddings_list)
        logger.info(f"Loaded {len(articles_memory)} articles into memory from Chroma")
        
    except Exception as e:
        logger.error(f"Failed to load articles into memory: {e}")
        articles_memory = []
        embeddings_memory = None

# Initialize in-memory data at startup
load_articles_to_memory()

# Function to generate query embedding using Ollama
def generate_query_embedding(query: str) -> Optional[np.ndarray]:
    """Generate embedding for a query using Ollama"""
    try:
        response = ollama.embeddings(model='nomic-embed-text', prompt=query)
        embedding = np.array(response['embedding'], dtype=np.float32)
        if len(embedding) != EMBEDDING_DIM:
            logger.error(f"Generated embedding has incorrect dimension: {len(embedding)}")
            return None
        return embedding
    except Exception as e:
        logger.error(f"Error generating query embedding: {e}")
        return None

def get_contextual_query(current_query: str) -> str:
    """Create an intent-aware contextual query by analyzing conversation history"""
    global conversation_history
    
    if not conversation_history:
        return current_query
    
    try:
        history_context = ""
        for i, (q, a) in enumerate(conversation_history[-MAX_HISTORY_LENGTH:]):
            history_context += f"User: {q}\nAssistant: {a}\n\n"
        
        prompt = f"""Given this conversation history and current query, please:
1. Identify the main intent and key entities in the current query
2. Determine if this query references previous conversation
3. Create an enhanced search query that captures the full intent

Conversation history:
{history_context}

Current query: {current_query}

Output only the enhanced search query that best captures the user's intent with any implicit references resolved.
"""
        
        response = ollama.generate(
            model='llama3:8b',
            prompt=prompt,
            options={
                'temperature': 0.1,
                'top_p': 0.9,
                'max_tokens': 200
            }
        )
        enhanced_query = response['response'].strip()
        
        logger.info(f"Generated intent-aware query: {enhanced_query[:100]}...")
        
        if not enhanced_query or len(enhanced_query) < 5:
            logger.warning("Intent extraction failed, using original query")
            return current_query
            
        return enhanced_query
        
    except Exception as e:
        logger.error(f"Error generating intent-aware query: {e}")
        context_parts = []
        for i, (q, _) in enumerate(conversation_history[-2:]):
            context_parts.append(f"Previous question: {q}")
        context_parts.append(f"Current question: {current_query}")
        return " ".join(context_parts)

def query_embeddings(query: str, use_context: bool = True, top_k: int = 10) -> List[Dict]:
    try:
        ollama.list()
        logger.info("Ollama server is running")
    except Exception as e:
        logger.error(f"Failed to connect to Ollama server: {e}")
        return []

    start_time = time.time()
    
    if use_context:
        contextual_query = get_contextual_query(query)
        logger.info(f"Using intent-aware query: {contextual_query[:100]}...")
    else:
        contextual_query = query
    
    query_embedding = generate_query_embedding(contextual_query)
    if query_embedding is None:
        logger.error("Failed to generate query embedding")
        return []
    
    if not articles_memory:
        logger.error("No articles available in memory")
        return []
    
    try:
        # Initialize Chroma client
        client = chromadb.PersistentClient(path=db_path)
        collection = client.get_collection(collection_name)
        
        # Query Chroma for top_k similar articles
        results = collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=top_k,
            include=['metadatas', 'documents', 'distances']
        )
        
        # Process results
        output = []
        for idx, (metadata, document, distance) in enumerate(
            zip(results['metadatas'][0], results['documents'][0], results['distances'][0])
        ):
            article = {
                'id': int(metadata.get('id', idx + 1)),
                'title': metadata.get('title', 'Unknown'),
                'url': metadata.get('url', 'Unknown'),
                'full_text': document,
                'publish_date': metadata.get('publish_date', 'Unknown'),
                'keyword': metadata.get('keyword', 'Unknown'),
                'author': metadata.get('author', 'Unknown'),
                'article_keywords': json.loads(metadata.get('article_keywords', '[]')),
                'similarity': 1 - distance  # Convert distance to similarity (cosine)
            }
            output.append(article)
        
        logger.info(f"Query processing took {time.time() - start_time:.2f} seconds")
        logger.info(f"Found {len(output)} relevant articles for query: {query}")
        return output
    
    except Exception as e:
        logger.error(f"Error querying Chroma: {e}")
        return []

def generate_response(query: str, articles: List[Dict]) -> Tuple[str, List[Dict]]:
    try:
        start_time = time.time()
        context_parts = []
        sources = []
        
        for i, a in enumerate(articles, 1):
            title = a.get('title', 'No Title')
            url = a.get('url', '#')
            text = a.get('full_text', '')[:1500]
            
            sources.append({
                'index': i,
                'title': title,
                'url': url
            })
            
            context_parts.append(f"Article {i}: {title}\nSource: {url}\n{text}")
        
        context = "\n\n".join(context_parts)
        
        history_context = ""
        if conversation_history:
            history_parts = []
            for q, a in conversation_history[-3:]:
                history_parts.append(f"User: {q}\nAssistant: {a}")
            history_context = "Previous conversation:\n" + "\n\n".join(history_parts) + "\n\n"
        
        prompt = f"""You are a helpful assistant that provides information based on news articles. 
When referencing information, include citation numbers [1], [2], etc. that correspond to the source articles.
Always reference your sources when providing facts. Always include URLs for your sources at the end of your response.

{history_context}
User: {query}

Here are relevant articles to help you answer:
{context}

Provide a helpful response with proper citations using [1], [2], etc. and include a "Sources:" section at the end with the article titles and URLs.
"""
        
        response = ollama.generate(
            model='llama3:8b',
            prompt=prompt,
            options={
                'temperature': 0.7,
                'top_p': 0.9,
                'max_tokens': 1500
            }
        )
        response_text = response['response'].strip()
        
        logger.info(f"Response generation took {time.time() - start_time:.2f} seconds")
        return response_text, sources
    except Exception as e:
        logger.error(f"Error generating response: {str(e)}")
        query_words = query.lower().split()
        
        fallback = f"I found some information that might help with your question about {' '.join(query_words[:3])}...\n\n"
        fallback += "Here are some topics I can share information about:\n"
        
        for i, a in enumerate(articles[:5], 1):
            title = a.get('title', 'No Topic')
            url = a.get('url', '#')
            fallback += f"{i}. {title}\n"
        
        fallback += "\nSources:\n"
        for i, a in enumerate(articles[:5], 1):
            title = a.get('title', 'No Topic')
            url = a.get('url', '#')
            fallback += f"[{i}] {title} - {url}\n"
            
        fallback += "\nWould you like to know more about any of these topics specifically?"
        return fallback, articles[:5]

def ensure_citations(response: str, sources: List[Dict]) -> str:
    """Make sure the response includes citations and a Sources section"""
    if not sources:
        return response
        
    if "Sources:" not in response:
        response += "\n\nSources:\n"
        for src in sources:
            response += f"[{src['index']}] {src['title']} - {src['url']}\n"
    
    return response

def main():
    global conversation_history
    
    print("\nContextual RAG News Search Assistant\n")
    
    if not articles_memory:
        print("No articles loaded in memory. Please check Chroma database.")
        return
    
    while True:
        query = input("\nWhat would you like to know? (type 'exit' to quit): ")
        if query.lower() in ['exit', 'quit', 'q']:
            print("\nThank you for using News Search Assistant")
            break
            
        total_start_time = time.time()
        
        print("\nSearching for relevant information...")
        results = query_embeddings(query, use_context=True, top_k=10)
        
        if not results:
            print("I couldn't find any relevant articles for your question.")
            continue

        print("Analyzing results to provide you with the best answer...")
        response, sources = generate_response(query, results)
        response = ensure_citations(response, sources)
        
        print("\n" + "─" * 80)
        print(response)
        print("─" * 80)
        
        conversation_history.append((query, response))
        
        if len(conversation_history) > MAX_HISTORY_LENGTH + 2:
            conversation_history = conversation_history[-MAX_HISTORY_LENGTH - 2:]
        
        if logger.level <= logging.DEBUG:
            print(f"\n[Debug: Total processing time: {time.time() - total_start_time:.2f} seconds]")
            
        if query.lower() == "clear history":
            conversation_history = []
            print("Conversation history cleared.")

if __name__ == "__main__":
    main()

/Users/pranavi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2025-05-06 18:47:37,615 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-05-06 18:47:37,926 - INFO - Loaded 50 articles into memory from Chroma



Contextual RAG News Search Assistant



2025-05-06 18:47:48,815 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
2025-05-06 18:47:48,817 - INFO - Ollama server is running
2025-05-06 18:47:48,818 - INFO - Using intent-aware query: pakistan...



Searching for relevant information...


2025-05-06 18:47:50,246 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
2025-05-06 18:47:50,263 - INFO - Query processing took 1.45 seconds
2025-05-06 18:47:50,264 - INFO - Found 10 relevant articles for query: pakistan


Analyzing results to provide you with the best answer...


2025-05-06 19:06:47,105 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-05-06 19:06:47,151 - INFO - Response generation took 1136.89 seconds



────────────────────────────────────────────────────────────────────────────────
Based on the articles provided, here is a helpful response:

India has taken a comprehensive strike against Pakistan following the deadly terror attack in Pahalgam that killed 26 people, most of them tourists [1]. The retaliatory measures include a blanket ban on all imports from Pakistan, suspension of mail exchanges, prohibition on Pakistani ships docking at Indian ports, and complete closure of Indian airspace to Pakistan-registered aircraft. This move has led to a significant escalation in diplomatic tensions between the two countries.

It's worth noting that India's foreign exchange reserves currently stand at over $688 billion, while Pakistan's reserves have barely crossed $15 billion [2]. This massive gap highlights decades of different policy choices, governance structures, and economic strategies pursued by the two countries.

In related news, Pakistani brides who entered India with short-term vi

In [2]:
!pip install langchain langchain-community langchain-chroma langchain-core

Defaulting to user installation because normal site-packages is not writeable
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 9.8 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
   ━━━━━━━━━━━━━━━━

### Modifications

In [6]:
import logging
import time
import json
from typing import List, Dict, Tuple
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain_core.documents import Document

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuration 
db_path = './chroma_db'  # Directory for Chroma persistent storage
collection_name = 'articles'
EMBEDDING_MODEL = 'nomic-embed-text'
LLM_MODEL = 'llama3:8b'
MAX_HISTORY_LENGTH = 5  # Maximum number of recent exchanges to keep

# Initialize LangChain components
embedding_function = OllamaEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    collection_name=collection_name,
    embedding_function=embedding_function,
    persist_directory=db_path
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
llm = Ollama(model=LLM_MODEL, temperature=0.7, top_p=0.9, num_ctx=4096)

# Set up conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    max_token_limit=1000
)

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant that provides information based on news articles. 
When referencing information, include citation numbers [1], [2], etc. that correspond to the source articles.
Always reference your sources when providing facts. Always include a Sources section at the end with the article titles and URLs.

Here are relevant articles to help you answer:
{context}

Provide a helpful response with proper citations using [1], [2], etc. and include a "Sources:" section at the end with the article titles and URLs."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

# Format context documents for the prompt
def format_docs(docs: List[Document]) -> str:
    context_parts = []
    for i, doc in enumerate(docs, 1):
        metadata = doc.metadata
        title = metadata.get('title', 'No Title')
        url = metadata.get('url', '#')
        text = doc.page_content[:1500]  # Limit text length
        context_parts.append(f"Article {i}: {title}\nSource: {url}\n{text}")
    return "\n\n".join(context_parts)

# Extract sources for citation
def extract_sources(docs: List[Document]) -> List[Dict]:
    sources = []
    for i, doc in enumerate(docs, 1):
        metadata = doc.metadata
        title = metadata.get('title', 'No Title')
        url = metadata.get('url', '#')
        sources.append({
            'index': i,
            'title': title,
            'url': url
        })
    return sources

# Define the RAG chain
def rag_chain_with_sources(query: str):
    # Retrieve relevant documents
    docs = retriever.invoke(query)
    
    # Format the context and extract sources
    context = format_docs(docs)
    sources = extract_sources(docs)
    
    # Get chat history
    chat_history = memory.load_memory_variables({})["chat_history"]
    
    # Run the LLM chain
    response = prompt_template.invoke({
        "context": context,
        "question": query,
        "chat_history": chat_history
    })
    
    llm_response = llm.invoke(response.to_string())
    
    # Ensure response has proper citations
    final_response = ensure_citations(llm_response, sources)
    
    return final_response, sources

def ensure_citations(response: str, sources: List[Dict]) -> str:
    """Ensure the response includes a Sources section with citations."""
    if not sources:
        return response
    if "Sources:" not in response:
        response += "\n\nSources:\n"
        for src in sources:
            response += f"[{src['index']}] {src['title']} - {src['url']}\n"
    return response

def main():
    print("\nContextual RAG News Search Assistant (Powered by LangChain)\n")

    # Check if vectorstore has articles
    try:
        sample_docs = vectorstore.get(limit=1)
        if not sample_docs['ids']:
            print("No articles loaded in Chroma database. Please populate the database.")
            return
        logger.info(f"Connected to Chroma database with articles")
    except Exception as e:
        print(f"Error connecting to Chroma database: {e}")
        return

    while True:
        query = input("\nWhat would you like to know? (type 'exit' to quit): ")
        print(query)
        if query.lower() in ['exit', 'quit', 'q']:
            print("\nThank you for using News Search Assistant")
            break

        total_start_time = time.time()
        print("\nSearching for relevant information...")

        try:
            # Run the RAG chain
            response, sources = rag_chain_with_sources(query)

            # Save to conversation history
            memory.save_context({"question": query}, {"output": response})

            # Trim conversation history
            history = memory.load_memory_variables({})["chat_history"]
            if len(history) > MAX_HISTORY_LENGTH * 2:  # Account for Human/AI pairs
                memory.chat_memory.messages = history[-MAX_HISTORY_LENGTH * 2:]

            print("\n" + "─" * 80)
            print(response)
            print("─" * 80)

            if logger.level <= logging.DEBUG:
                print(f"\n[Debug: Total processing time: {time.time() - total_start_time:.2f} seconds]")

            if query.lower() == "clear history":
                memory.clear()
                print("Conversation history cleared.")

        except Exception as e:
            logger.error(f"Error processing query: {e}")
            print("An error occurred while processing your query. Please try again.")

if __name__ == "__main__":
    main()

2025-05-07 11:13:40,648 - INFO - Connected to Chroma database with articles



Contextual RAG News Search Assistant (Powered by LangChain)


Searching for relevant information...


2025-05-07 11:13:56,008 - INFO - Backing off send_request(...) for 0.8s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Read timed out. (read timeout=15))



────────────────────────────────────────────────────────────────────────────────
The situation between India and Pakistan is escalating rapidly. The recent terrorist attack in Pahalgam, Jammu and Kashmir, has led to a significant deterioration in diplomatic relations between the two countries.

Pakistan has threatened that any attempt to limit waters from the Indus would be regarded as "an act of war" [1]. This comes after India suspended the Indus Waters Treaty of 1960, which would severely reduce Pakistan's water supply [2].

In response to the attack, India has demanded that Italy cut financing to Pakistan and also approached FATF (Financial Action Task Force) to include Pakistan in the grey list [3]. Additionally, India has begun work on hydroelectric projects to boost reservoir holding capacity at two dams in the Himalayan region of Kashmir [4].

Pakistan's Prime Minister Shehbaz Sharif has said that the country is prepared "for national defence" after conducting a second missile

### Query updates

In [5]:
import logging
import time
from typing import List, Dict
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain_core.documents import Document
import os
import shutil

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuration
db_path = './chroma_db'
collection_name = 'articles'
EMBEDDING_MODEL = 'nomic-embed-text'
LLM_MODEL = 'llama3:8b'
MAX_HISTORY_LENGTH = 5

# Initialize Chroma vectorstore
def initialize_vectorstore():
    """Initialize Chroma vectorstore with error handling."""
    try:
        vectorstore = Chroma(
            collection_name=collection_name,
            embedding_function=embedding_function,
            persist_directory=db_path
        )
        logger.info("Successfully initialized Chroma vectorstore")
        # Verify database has articles
        sample_docs = vectorstore.get(limit=1)
        if not sample_docs['ids']:
            logger.warning("Chroma database is empty")
            return None
        return vectorstore
    except Exception as e:
        logger.error(f"Failed to initialize Chroma database: {e}")
        print(f"Error initializing Chroma database: {e}")
        print("Attempting to reset and recreate collection...")
        try:
            # Backup and reset database
            if os.path.exists(db_path):
                backup_path = f"{db_path}.bak_{int(time.time())}"
                shutil.move(db_path, backup_path)
                logger.info(f"Backed up database to {backup_path}")
            vectorstore = Chroma(
                collection_name=collection_name,
                embedding_function=embedding_function,
                persist_directory=db_path
            )
            logger.info("Recreated Chroma database and collection")
            # Check if new database has articles
            sample_docs = vectorstore.get(limit=1)
            if not sample_docs['ids']:
                logger.warning("New Chroma database is empty")
                return None
            return vectorstore
        except Exception as e2:
            logger.error(f"Failed to recreate Chroma database: {e2}")
            return None

# Initialize components
embedding_function = OllamaEmbeddings(model=EMBEDDING_MODEL)
vectorstore = initialize_vectorstore()

# Only proceed with initializations if vectorstore is valid
if vectorstore is None:
    print("Failed to initialize Chroma database")
    exit(1)

# Initialize remaining components
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
llm = Ollama(model=LLM_MODEL, temperature=0.7, top_p=0.9, num_ctx=4096)

# Set up conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    max_token_limit=1000
)

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant that provides information based on news articles. 
When referencing information, include citation numbers [1], [2], etc. that correspond to the source articles.
Always reference your sources when providing facts. Always include a Sources section at the end with the article titles and URLs.

Here are relevant articles to help you answer:
{context}

Provide a helpful response with proper citations using [1], [2], etc. and include a "Sources:" section at the end with the article titles and URLs."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

# Format context documents for the prompt
def format_docs(docs: List[Document]) -> str:
    context_parts = []
    for i, doc in enumerate(docs, 1):
        metadata = doc.metadata
        title = metadata.get('title', 'No Title')
        url = metadata.get('url', '#')
        text = doc.page_content[:1500]
        context_parts.append(f"Article {i}: {title}\nSource: {url}\n{text}")
    return "\n\n".join(context_parts)

# Extract sources for citation
def extract_sources(docs: List[Document]) -> List[Dict]:
    sources = []
    for i, doc in enumerate(docs, 1):
        metadata = doc.metadata
        title = metadata.get('title', 'No Title')
        url = metadata.get('url', '#')
        sources.append({
            'index': i,
            'title': title,
            'url': url
        })
    return sources

# Define the RAG chain
def rag_chain_with_sources(query: str):
    # Retrieve relevant documents
    docs = retriever.invoke(query)
    
    # Format the context and extract sources
    context = format_docs(docs)
    sources = extract_sources(docs)
    
    # Get chat history
    chat_history = memory.load_memory_variables({})["chat_history"]
    
    # Run the LLM chain
    response = prompt_template.invoke({
        "context": context,
        "question": query,
        "chat_history": chat_history
    })
    
    llm_response = llm.invoke(response.to_string())
    
    # Ensure response has proper citations
    final_response = ensure_citations(llm_response, sources)
    
    return final_response, sources

def ensure_citations(response: str, sources: List[Dict]) -> str:
    """Ensure the response includes a Sources section with citations."""
    if not sources:
        return response
    if "Sources:" not in response:
        response += "\n\nSources:\n"
        for src in sources:
            response += f"[{src['index']}] {src['title']} - {src['url']}\n"
    return response

def main():
    print("\nContextual RAG News Search Assistant (Powered by LangChain)\n")

    # Check if vectorstore has articles
    try:
        sample_docs = vectorstore.get(limit=1)
        if not sample_docs['ids']:
            print("No articles loaded in Chroma database. Please populate the database with articles.")
            print("Example: Use vectorstore.add_documents([Document(page_content='Text', metadata={'title': 'Title', 'url': 'URL'})])")
            print("If you have a script like 'articles_chroma.py', run it to populate the database, or restore a backup.")
            return
        logger.info(f"Connected to Chroma database with articles")
    except Exception as e:
        logger.error(f"Error accessing Chroma database: {e}")
        print("Error accessing Chroma database. Please ensure './chroma_db' is valid and populated.")
        return

    while True:
        query = input("\nWhat would you like to know? (type 'exit' to quit): ")
        print(f"user query: {query}")
        if query.lower() in ['exit', 'quit', 'q']:
            print("\nThank you for using News Search Assistant")
            break

        total_start_time = time.time()
        print("\nSearching for relevant information...")

        try:
            # Run the RAG chain
            response, sources = rag_chain_with_sources(query)

            # Save to conversation history
            memory.save_context({"question": query}, {"output": response})

            # Trim conversation history
            history = memory.load_memory_variables({})["chat_history"]
            if len(history) > MAX_HISTORY_LENGTH * 2:
                memory.chat_memory.messages = history[-MAX_HISTORY_LENGTH * 2:]

            print("\n" + "─" * 80)
            print(response)
            print("─" * 80)

            if logger.level <= logging.DEBUG:
                print(f"\n[Debug: Total processing time: {time.time() - total_start_time:.2f} seconds]")

            if query.lower() == "clear history":
                memory.clear()
        except Exception as e:
            logger.error(f"Error processing query: {e}")
            print("An error occurred while processing your query. Please try again.")

if __name__ == "__main__":
    main()

2025-05-07 20:58:53,925 - INFO - Successfully initialized Chroma vectorstore
2025-05-07 20:58:53,953 - INFO - Connected to Chroma database with articles



Contextual RAG News Search Assistant (Powered by LangChain)

user query: ipl

Searching for relevant information...

────────────────────────────────────────────────────────────────────────────────
I'm happy to help! However, it seems like you didn't ask a specific question or provide any context about what you're looking for. If you could provide more information or clarify your query, I'll do my best to assist you.

In the meantime, here are some interesting articles from recent news sources:

* "We missed you': Rohit Sharma presents T20 World Cup winning ring to Mohammad Siraj" [1]
* "India makes Chenab run dry, for now" [2]
* "India demands Italy cut financing to Pakistan" [3]
* "Delhi National Lok Adalat: Check date, token registration, eligibility, documents required to settle your traffic challans" [4]
* "Why startup investors are eyeing sports teams" [5]
* "Stock market update: Nifty IT index advances 0.28%" [6]
* "Hyderabad: Miss World 2025 contestants to attend IPL match as 